In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
from pathlib import Path

# Configurable parts
SOURCE = "dukascopy"
SYMBOL = "usdjpy"
MINUTES = 1
START_DATE = "2021-01-01"
END_DATE = "2024-12-31"

# Build base name
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
NORMALIZED_DIR = BASE_DIR / "normalized"
SCALER_DIR = BASE_DIR / "scalers"

# Final paths
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_processed.pkl"
NORMALIZED_FILE_PATH = NORMALIZED_DIR / f"{RESAMPLED_NAME}_normalized.pkl"
STD_SCALER_PATH = SCALER_DIR / f"{RESAMPLED_NAME}_standard_scaler.pkl"
MINMAX_SCALER_PATH = SCALER_DIR / f"{RESAMPLED_NAME}_minmax_scaler.pkl"

In [3]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
df.head()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-03 23:03:00,103.2325,103.2465,103.2290,103.2440,476970.000148,0.014912,13.075211,0.0110,0.000107,0.000107,...,0.965926,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025
2021-01-03 23:04:00,103.2440,103.2480,103.2345,103.2395,267090.000629,0.011971,12.495345,-0.0045,-0.000044,-0.000044,...,0.965926,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025
2021-01-03 23:05:00,103.2380,103.2505,103.2355,103.2425,214629.999399,0.012319,12.276676,0.0030,0.000029,0.000029,...,0.965926,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025
2021-01-03 23:06:00,103.2425,103.2475,103.2420,103.2450,208259.999037,0.013018,12.246547,0.0025,0.000024,0.000024,...,0.965926,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025
2021-01-03 23:07:00,103.2460,103.2480,103.2330,103.2335,299720.000982,0.011103,12.610607,-0.0115,-0.000111,-0.000111,...,0.965926,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025


In [4]:
df.describe()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
count,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,...,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06
mean,1.333537e+02,1.333646e+02,1.333426e+02,1.333537e+02,4.642408e+05,7.100295e-03,1.244988e+01,3.632761e-05,2.960179e-07,2.832624e-07,...,-6.506053e-03,2.024515e+00,3.497092e-01,-5.183642e-02,1.570614e+01,2.776572e-03,-2.243766e-02,6.519712e+00,-4.318901e-03,-4.816533e-03
std,1.675189e+01,1.675451e+01,1.674920e+01,1.675189e+01,4.806075e+05,9.693782e-03,1.271653e+00,2.218189e-02,1.597076e-04,1.597288e-04,...,7.070214e-01,1.478414e+00,5.229292e-01,7.756043e-01,8.772717e+00,7.140145e-01,6.997662e-01,3.439120e+00,7.075227e-01,7.066615e-01
min,1.026025e+02,1.026055e+02,1.025925e+02,1.026025e+02,0.000000e+00,1.000000e-03,0.000000e+00,-2.065000e+00,-1.380804e-02,-1.390426e-02,...,-1.000000e+00,0.000000e+00,-7.818315e-01,-9.009689e-01,1.000000e+00,-9.987165e-01,-9.948693e-01,1.000000e+00,-1.000000e+00,-1.000000e+00
25%,1.145900e+02,1.145960e+02,1.145835e+02,1.145895e+02,1.216400e+05,3.960000e-03,1.170883e+01,-7.500000e-03,-5.820362e-05,-5.820531e-05,...,-7.071068e-01,1.000000e+00,0.000000e+00,-9.009689e-01,8.000000e+00,-7.247928e-01,-7.587581e-01,4.000000e+00,-8.660254e-01,-8.660254e-01
50%,1.365335e+02,1.365470e+02,1.365200e+02,1.365335e+02,3.062200e+05,5.703704e-03,1.263206e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,-1.836970e-16,2.000000e+00,4.338837e-01,-2.225209e-01,1.600000e+01,-2.449294e-16,-5.064917e-02,7.000000e+00,-2.449294e-16,-1.836970e-16
75%,1.477020e+02,1.477140e+02,1.476905e+02,1.477020e+02,6.486800e+05,7.272727e-03,1.338270e+01,8.000000e-03,5.982608e-05,5.982429e-05,...,7.071068e-01,3.000000e+00,7.818315e-01,6.234898e-01,2.300000e+01,7.247928e-01,6.889669e-01,1.000000e+01,5.000000e-01,5.000000e-01
max,1.619500e+02,1.619510e+02,1.619435e+02,1.619495e+02,5.114110e+06,3.740000e-01,1.544751e+01,1.579500e+00,1.228753e-02,1.221265e-02,...,1.000000e+00,6.000000e+00,9.749279e-01,1.000000e+00,3.100000e+01,9.987165e-01,1.000000e+00,1.200000e+01,1.000000e+00,1.000000e+00


In [5]:
FEATURES_COLS = [
    # Basic Data
    'close_log_return',
    'log_volume',
    'spread',

    # Other
    'ret_mean_5',
    'ret_mean_10'
]
TIME_FEATURE_COLS = [
    'hour_cos',
    'dow_cos',
    'dom_cos',
    'month_cos',
]
TA_MIN_MAX = [
    # TA
    'rsi_14',
]
TA_STD = [
    'ema_21',
    'sma_50',
    'atr_14',
    
    'bb_upper',
    'bb_lower',
    'bb_mavg',
    'bb_width',

    'donchian_upper',
    'donchian_lower',
    'donchian_mid',

    'stoch_k',
    'stoch_d',

    'macd',
    'macd_signal',
    'macd_diff',
]

In [6]:
COLS_TO_STD = FEATURES_COLS + TA_STD
COLS_TO_MIN_MAX = TA_MIN_MAX

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

In [8]:
df[COLS_TO_STD] = standard_scaler.fit_transform(df[COLS_TO_STD])
df[COLS_TO_MIN_MAX] = minmax_scaler.fit_transform(df[COLS_TO_MIN_MAX])

In [9]:
df.describe()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
count,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,...,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06,1.480293e+06
mean,1.333537e+02,1.333646e+02,1.333426e+02,1.333537e+02,4.642408e+05,-4.915214e-17,-9.830429e-17,3.632761e-05,2.960179e-07,-6.950421e-18,...,-6.506053e-03,2.024515e+00,3.497092e-01,-5.183642e-02,1.570614e+01,2.776572e-03,-2.243766e-02,6.519712e+00,-4.318901e-03,-4.816533e-03
std,1.675189e+01,1.675451e+01,1.674920e+01,1.675189e+01,4.806075e+05,1.000000e+00,1.000000e+00,2.218189e-02,1.597076e-04,1.000000e+00,...,7.070214e-01,1.478414e+00,5.229292e-01,7.756043e-01,8.772717e+00,7.140145e-01,6.997662e-01,3.439120e+00,7.075227e-01,7.066615e-01
min,1.026025e+02,1.026055e+02,1.025925e+02,1.026025e+02,0.000000e+00,-6.293000e-01,-9.790310e+00,-2.065000e+00,-1.380804e-02,-8.705102e+01,...,-1.000000e+00,0.000000e+00,-7.818315e-01,-9.009689e-01,1.000000e+00,-9.987165e-01,-9.948693e-01,1.000000e+00,-1.000000e+00,-1.000000e+00
25%,1.145900e+02,1.145960e+02,1.145835e+02,1.145895e+02,1.216400e+05,-3.239495e-01,-5.827427e-01,-7.500000e-03,-5.820362e-05,-3.661745e-01,...,-7.071068e-01,1.000000e+00,0.000000e+00,-9.009689e-01,8.000000e+00,-7.247928e-01,-7.587581e-01,4.000000e+00,-8.660254e-01,-8.660254e-01
50%,1.365335e+02,1.365470e+02,1.365200e+02,1.365335e+02,3.062200e+05,-1.440708e-01,1.432674e-01,0.000000e+00,0.000000e+00,-1.773397e-03,...,-1.836970e-16,2.000000e+00,4.338837e-01,-2.225209e-01,1.600000e+01,-2.449294e-16,-5.064917e-02,7.000000e+00,-2.449294e-16,-1.836970e-16
75%,1.477020e+02,1.477140e+02,1.476905e+02,1.477020e+02,6.486800e+05,1.778798e-02,7.335496e-01,8.000000e-03,5.982608e-05,3.727635e-01,...,7.071068e-01,3.000000e+00,7.818315e-01,6.234898e-01,2.300000e+01,7.247928e-01,6.889669e-01,1.000000e+01,5.000000e-01,5.000000e-01
max,1.619500e+02,1.619510e+02,1.619435e+02,1.619495e+02,5.114110e+06,3.784899e+01,2.357277e+00,1.579500e+00,1.228753e-02,7.645691e+01,...,1.000000e+00,6.000000e+00,9.749279e-01,1.000000e+00,3.100000e+01,9.987165e-01,1.000000e+00,1.200000e+01,1.000000e+00,1.000000e+00


In [10]:
print(df[COLS_TO_STD].mean())         # Should be ~0
print(df[COLS_TO_STD].std(ddof=0))

close_log_return   -6.950421e-18
log_volume         -9.830429e-17
spread             -4.915214e-17
ret_mean_5          4.147212e-18
ret_mean_10         5.376016e-18
ema_21             -5.505040e-16
sma_50              0.000000e+00
atr_14              1.966086e-17
bb_upper           -3.932172e-16
bb_lower            3.145737e-16
bb_mavg             7.077909e-16
bb_width            2.949129e-17
donchian_upper     -7.864343e-16
donchian_lower      6.291475e-16
donchian_mid        7.077909e-16
stoch_k            -8.048664e-17
stoch_d            -7.741463e-17
macd               -2.150406e-18
macd_signal         7.526422e-18
macd_diff          -7.587622e-18
dtype: float64
close_log_return    1.0
log_volume          1.0
spread              1.0
ret_mean_5          1.0
ret_mean_10         1.0
ema_21              1.0
sma_50              1.0
atr_14              1.0
bb_upper            1.0
bb_lower            1.0
bb_mavg             1.0
bb_width            1.0
donchian_upper      1.0
donchian_lowe

In [11]:
import joblib
joblib.dump(standard_scaler, STD_SCALER_PATH)
joblib.dump(minmax_scaler, MINMAX_SCALER_PATH)
df.to_pickle(NORMALIZED_FILE_PATH)